In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

In [ ]:
!pip install -q langchain_google_genai= langchain-community langchain-core requests duckduckgo-search ddgs google-ai-generativelanguage

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import requests

In [9]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [11]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'

  response = requests.get(url)

  return response.json()

In [12]:
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=0)

In [13]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [14]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


In [15]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [16]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [17]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
I need to first find the capital of Madhya Pradesh, and then use that information to find the current weather conditions in that city.
Action: duckduckgo_search
Action Input: "capital of Madhya Pradesh"2 weeks ago - Its capital is Bhopal . Other major cities includes Indore, Gwalior, Ujjain, Jabalpur and Sagar. Madhya Pradesh is the second largest Indian state by area and the fifth largest state by population with over 72 million residents. It borders the states of Rajasthan to the northwest, Uttar Pradesh ... September 12, 2025 - Bhopal (Hindi: Bhōpāl, pronounced [bʱoːpaːlᵊ] ⓘ) is the capital city of the Indian state of Madhya Pradesh and the administrative headquarters of both Bhopal district and Bhopal division. It is known as the City of Lakes, due to presence of various natural and artificial lakes near ... 1 week ago - Indore (/ɪnˈdɔːr/ ⓘ; ISO: Indaura, Hindi: [ɪn̪d̪ɔːr]) is the largest and most populous city in the Indian state of Madhya P

In [18]:
response['output']

'The capital of Madhya Pradesh is Bhopal. I am unable to retrieve the current weather conditions for Bhopal at this time.'